In this notebook, we will do some basic data analyses. As a basis, we will use a synthetic data set containing biographical and medicinal information of 10.000 persons.

# 0. Python and data primer

## 0.1. Python primer

In this section we will provide a concise overview of the Python language, to enable you to get started. The '#' sign is used to start a comment in Python, that lasts until the end of the line. The 'print()' command can be used to print output. The 'type()' command shows the type of its argument. Python indexing always starts at 0. For more information about Python, please refer to https://docs.python.org/3/reference/index.html.

### Primitive types
First, lets go over some primitive types in Python: integers (whole numbers), floats (real numbers), booleans (true/false) and strings (text).

In [ ]:
# integers
a = 1
print(type(a))
print(a)

In [ ]:
# floats
b = 1.0
type(b)
print(b)

In [ ]:
# booleans
c = True
print(type(c))
print(c)
d = a == 2
print(d)

In [ ]:
# string
s = 'We are the knights who say "Ni!"'
print(type(s))
print(s)
s2 = s + ' And we demand a shrubbery!'
print(s2)

### Built-in basic data types

Python has a number of built-in data types, including lists, dictionaries and sets. We will briefly go over the uses and syntax of each of these.

A *list* is an ordered sequence of values, not necessarily of the same type. Lists can be initialized via square brackets [].

In [ ]:
l = [1, 'blah', 2.0, True]
print(type(l))
print(l)

# iterate over list
for i in l:
    print(i)
    
# add element to a list
l.append(5)
print(l)

# list slicing: print the first 3 elements
print(l[0:3])

# length of the list: use the len() function
print(len(l))

A *dictionary* (dict) is a container that maps (unique) keys to values, and is typically initialized using curly brackets {}.

In [ ]:
d = {'a': 1, 'b': 2}
print(type(d))
print(d)

# get item 'a' from the dictionary
print(d['a'])

# length of the dictionary
print(len(d))

# iterate over the dictionary
# note that the ordering is not necessarily as initialized
for k, v in d.items():
    print('key %s mapped to value %d' % (k, v))

A *set* is a container for distinct values, and can be initialized via the *set()* function.

In [ ]:
s1 = set([1, 2, 3, 3])
print(type(s1))
print(s1)

s2 = set([3, 4, 5])
print(s2)

# set union
s3 = s1.union(s2)
print(s3)

# set difference
s4 = s1.difference(s2)
print(s4)

### Various useful functions

The 'help()' function can be used to open the documentation of a certain function or data type.

In [ ]:
# 1.0 represents a float object
help(1.0)

In [ ]:
# map() is a built-in function
help(map)

List comprehensions provide an elegant way to create or transform iterables.

In [ ]:
# to see what range() does, use help(range)
a = [5 * i for i in range(5)]
print(a)
b = [x - 1 for x in a]
print(b)

The next step is importing libraries and/or data. Python has a wealth of libraries that provide well-documented functions we can use for data analysis. In particular, we will use 'NumPy' (numerical support, esp. linear algebra), 'scikit-learn' (machine learning), 'pandas' (statistics) and matplotlib (plotting). A library can be loaded using the 'import' command and renamed via 'import x as y'. To call function x from library y, we use y.x().

In [ ]:
import numpy as np # numpy
import sklearn # scikit-learn
import pandas as pd # pandas

#%matplotlib notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt # matplotlib
matplotlib.style.use('ggplot')

## 0.2 The data set

We will use a synthetic data set based on health expenditure data. The data is organized as a matrix, and contains biographical information, class labels and information about drug purchases.

For your convenience, we created a small library for loading the data and some convenience functions. We will load the library as 'ibd4h'.

In [ ]:
import ibd4health as ibd4h
data = ibd4h.data
features = ibd4h.features
labels = ibd4h.labels
colidx = ibd4h.colidx

The following variables are important:
- **data**: the data matrix, rows correspond to patients, columns to features
- **features**: the list of features (corresponding to columns in ibd4h.data)
- **labels**: the labels for patients (corresponding to rows, True: diabetic, False: non-diabetic)
- **colidx**: a dictionary to facilitate retrieving columns from their string definitions

In [ ]:
print("Data set contains info on %d patients with %d features." 
      % data.shape)
print("Feature list has %d entries." 
      % features.shape)
print("Label list has %d entries, of which %d are positive." 
      % (labels.shape[0], sum(labels)))

The features in this data set include age, gender the volume of drugs purchased by the patient. Lets have a look at the first 3 features.

In [ ]:
for idx, feat in enumerate(features[:3]):
    print('feature %d: %s' % (idx, feat))

Drug volumes are categorized via codes of the anatomical therapeutic chemical (ATC) classification system (cfr. http://www.whocc.no/atc_ddd_index/ for details). You can find a description of ATC code XXX in the data set at http://www.whocc.no/atc_ddd_index/?showdescription=yes&code=XXX. For example, information on the third feature 'A01AA01', can be found at http://www.whocc.no/atc_ddd_index/?showdescription=yes&code=A01AA01.

The data matrix can be indexed via `data[rowidx, colidx]`. For example, the age of the first patient:

In [ ]:
ageidx = colidx['age']
age = data[0, ageidx]
print(age)

For slicing, replace either rowidx or colidx with ":". For example, to get a vector of all ages, we can use:

In [ ]:
ages = data[:, ageidx]
print(ages.shape)
print('Average age: %1.3f' % np.mean(ages))
print('Minimum age: %1.3f' % np.min(ages))
print('Maximum age: %1.3f' % np.max(ages))

# 1. Exploratory analysis

In [ ]:
genderidx = colidx['gender']
# create a list of gender indicators via list comprehension
genders = [x > 0 for x in data[:, genderidx]]

# define gender representation
male = False
female = True

num_males = genders.count(male)
num_females = genders.count(female)
print('Data set contains %d males and %d females.' % (num_males, num_females))

# gender distribution in diabetics
positive_rowidx = [idx for idx, label in enumerate(labels) if label]
print(len(positive_rowidx)) # sanity check: length of this list should be 5000
positive_genders = [x > 0 for x in data[positive_rowidx, genderidx]]

num_positive_males = positive_genders.count(male)
num_positive_females = positive_genders.count(female)
print('Data set contains %d positive males and %d positive females.' % (num_positive_males, num_positive_females))

In [ ]:
plt.hist(data[:, ageidx], bins=50)
plt.xlabel('age')
plt.ylabel('frequency')
plt.title('Age histogram')

In [ ]:
# create a mask to identify males and females in the data
male = data[:, colidx['gender']] > 0
female = np.invert(male)
print(female[:5])
print(male[:5])

# create vectors of ages
male_ages = data[male, ageidx]
female_ages = data[female, ageidx]

# plot histograms of age vectors
plt.hist((male_ages, female_ages), bins=50, histtype='step')
plt.xlabel('age')
plt.ylabel('frequency')
plt.legend(['female (mean: %1.3f)' % np.mean(female_ages),
            'male (mean: %1.3f)' % np.mean(male_ages)],
          loc='lower center')
plt.title('Age histogram per gender')

# 2. Simple predictive modelling

# 3. Evaluating performance

![Train test split](pics/fig_split-1.png)
![5 fold Cross validation](pics/fig_cv-1.png)

# 4. Feature selection

We will work with some feature selection methods in order to remove attributes that may not be necessary for the predictive models. Let's have a look again at the data rapidly. This time we use a pandas DataFrame and then printing the first rows as well as a description for each feature.

In [ ]:
# Create the data frame
df = pd.DataFrame(data,columns=features)
# Print the first rows
print (df.head())
# Print a description of the variables
print (df.describe())

#df['gender'].astype(int).unique()

# A simple technique for feature selection is to remove the attributes with low or zero variance.
# For this purpose we will Scikit-learn
from sklearn.feature_selection import VarianceThreshold

# here we can specify a threshold, default is zero variance

var_selector = VarianceThreshold(0.0)
new_data = var_selector.fit_transform(data)

print ("Reduced dataset contains {0:d} features".format(new_data.shape[1]))

So, the dataset contains a lot of features with zero variance. Removing these features can speed up the learning process.

Now, let's try a simple method which orders the features according to their importances calculated for a certain scoring function. For example, that could be a statistical test (e.g. $\chi^2$) or even another machine learning model (e.g. a linear model with $L_1$ regularization). 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,chi2,SelectFromModel

# helper function for cross-validating a classifier
# we use a fixed seed in order to compare classifiers
# across the same folds
def crossValidateClassifier(X,y,clf):
    cross_val = cross_validation.KFold(len(y),n_folds=5,shuffle=True,random_state=224441)
    cv_score = cross_validation.cross_val_score(clf,X,y,scoring='roc_auc',cv=cross_val)
    return np.mean(cv_score),np.std(cv_score)

# We use a non-linear model namely Random Forest which 
# builds multiple decision trees
# ==== Marc, I suposse here we will use parameters we optimize in previously ====
model = RandomForestClassifier(n_estimators=100,max_depth=35,n_jobs=3)

# We will try several values for the selected features
k_values = np.arange(100,1000,100)
mean = []
stds = []
for k in k_values:
    X_new = SelectKBest(chi2, k=k).fit_transform(data, labels)
    m,std = crossValidateClassifier(X_new,labels,model)
    mean.append(m)
    stds.append(std)

m,std = crossValidateClassifier(data,labels,model)

plt.errorbar(k_values,mean,yerr=stds,color="blue", fmt='-o',label="x2")
plt.axhline(y=m,color='red',label="All features")
plt.xlabel('Number of features')
plt.ylabel('Roc-auc score')
plt.title('#features vs. Roc-auc score')
plt.legend(loc='best')
plt.xlim(50,950)

We will use now another predictive model in order to select a number of features and then feed them to the learner. Scikit-learn provides a wrapper SelectFromModel for this purpose.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# A parameter to be tuned for the feature selector
c_values = [0.001,0.01,0.1,1,10,100,1000]

mean = []
stds = []
shapes = []
for c in c_values:
    # We use a linear model with L1-regularization as feature selector
    selector = SelectFromModel(LinearSVC(C=c, penalty="l1", dual=False),prefit=False)
    X_new = selector.fit_transform(data,labels)
    print ("Selected {0:d} features, C = {1:f}".format(X_new.shape[1],c))
    mn,std = crossValidateClassifier(X_new,labels,model)
    shapes.append(X_new.shape[1])
    mean.append(mn)
    stds.append(std)

plt.errorbar(shapes,mean,yerr=stds,color="blue", fmt='-o',label="L1 feature selection")
plt.axhline(y=m,color='red',label="All features")
plt.xlabel('Number of features')
plt.ylabel('Roc-auc score')
plt.title('#features vs. Roc-auc score')
plt.legend(loc='best')
plt.xlim(340,880)

# 5. Constructing a machine learning pipeline

So, we have seen sofar how to pre-process a dataset, apply features selection methods, tune parameters and train predictive models. Usually, all these steps are packed in a single pipeline in order to replicate easilly the experiments with different settings.

In [ ]:
# We first import the method from Scikit-learn
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn import metrics

# We define a sequence of steps
# Let's start with a fetures selection method followed by a random-forest learner

transformers = [('select_k_best',SelectKBest(chi2)),("rf_model",RandomForestClassifier())]
pipeline = Pipeline(transformers)

# set some parameters for grid searching
# to do that we use the following convention: nameoftransformer__nameofparameter=[list of values]

params = dict(select_k_best__k=np.arange(100,400,100), rf_model__n_estimators=[50,100,300], 
              rf_model__max_depth=[10, 20,30])

# Do a grid search with 5-cv. Note that this results to 27*5 fits. So, it may take some time for large datasets
# Let's split the data to speed-up things

X_train,X_test,y_train,y_test = cross_validation.train_test_split(data,labels,test_size=0.7,random_state=51)

tuned_pipeline = GridSearchCV(pipeline, param_grid=params, n_jobs=3, refit=True, cv=5).fit(X_train, y_train)

# Print the best parameters found
print (tuned_pipeline.best_params_)

# Test these parameters. The pipeline is a convenient way to apply all the steps in new data.
# We simply have to call the predict method as in the case of other estimators.

print  ("Accuracy: {0:f}".format(metrics.accuracy_score(y_test, tuned_pipeline.predict(X_test))))

Exercise: repeat the same procedure by adding a further step of feature selection which removes the low variance features.